# Analysis of the databe of CALIOPE

```markdown
The data is from CALIOPE, a model that predicted the concentrations we have in the data. We will use this data to train our model, which will help increase the resolution by first predicting the 4 stations that are not provided within the database.
When we faced the problem, the biggest issue was the volume of the dataframe. Given the hardware circumstances of our local PC, it was inevitable to split the data, although we are aware that this will result in some loss of information.
```

## Prepare the data

The code reads multiple NetCDF files from the directory provided for us, extracts 1000 random rows from each file, concatenates the extracted data into a single DataFrame, and saves it as a CSV file. A big problem with this method of extraction of datas is that some of the points will be missed in our dataset of train, which will cause a great bias in the future model training.

In [ ]:
import os
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt


def extract_random_rows_v3(nc_data, num_rows=1000):
    """ Extraer 1000 filas aleatorias de las columnas time, lat, lon y sconcno2. """
    # Convertir el dataset de xarray a DataFrame y seleccionar solo las columnas necesarias
    df = nc_data[['time', 'lat', 'lon', 'sconcno2']].to_dataframe().reset_index()

    # Seleccionar 1000 filas aleatorias
    if len(df) > num_rows:
        sampled_df = df.sample(n=num_rows, random_state=42).reset_index(drop=True)
    else:
        sampled_df = df

    return sampled_df


# Ruta de la carpeta donde se encuentran los archivos .nc
directorio = r'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/'

# Listar los archivos .nc de la carpeta
archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.nc')]

# Lista para almacenar todos los DataFrames generados
lista_dataframes = []

# Recorre todos los archivos .nc de la carpeta
for archivo in archivos:
    try:
        ruta_archivo = os.path.join(directorio, archivo)
        print(f"Procesando archivo: {archivo}")
        
        # Abrir el archivo NetCDF
        nc_data = xr.open_dataset(ruta_archivo)
        
        # Extraer 1000 filas aleatorias
        sampled_data = extract_random_rows_v3(nc_data)
        
        # Agregar el DataFrame a la lista
        lista_dataframes.append(sampled_data)
        
        # Cerrar el dataset para liberar memoria
        nc_data.close()
        
        print(f"Archivo procesado correctamente: {archivo} | Filas extraídas: {sampled_data.shape[0]}")
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")

# Concatenar todos los DataFrames
data_final = pd.concat(lista_dataframes, ignore_index=True)
data_final = data_final.drop(columns=['x', 'y', 'lev'])
# Guardar el DataFrame final como CSV
output_path = 'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/NO2_final.csv'
data_final.to_csv(output_path, index=False)
print(f"Archivo CSV guardado en: {output_path}")

# Mostrar las primeras filas para verificar
print("Vista previa de los primeros registros:")
print(data_final.head())

# Mostrar información general del DataFrame
print(f"Total de filas: {data_final.shape[0]}")
print(f"Columnas del DataFrame: {data_final.columns}")

# Mostrar una vista previa de los primeros 5 registros de cada conjunto
""" import ace_tools as tools
tools.display_dataframe_to_user(name="Sampled Data from File 1 (Final V2)", dataframe=sampled_data_1_final_v2.head())
 """

In [1]:

NO2 = pd.read_csv('C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/NO2_final.csv')
NO2['time'] = pd.to_datetime(NO2['time'])
print(NO2.dtypes)

print(NO2.describe())
for i in NO2.columns:
    print(NO2[i].value_counts())

filas_con_valor = NO2.loc[NO2['sconcno2'] == 9.969210e+36]

print("Filas con sconcno2 = 9.969210e+36:")
print(filas_con_valor)

df_filtrado = NO2.loc[~NO2['time'].dt.strftime('%Y-%m-%d').isin(['2023-08-20', '2023-08-21'])]

print("DataFrame después de eliminar las filas con fechas 20 y 21 de agosto:")
print(df_filtrado)
for i in df_filtrado.columns:
    print(df_filtrado[i].value_counts())
print(df_filtrado.describe())

time        datetime64[ns]
lat                float64
lon                float64
sconcno2           float64
dtype: object
                                time            lat            lon  \
count                         332000  332000.000000  332000.000000   
mean   2023-07-02 15:15:34.048192512      41.638951       1.717181   
min              2023-01-01 00:00:00      40.276993       0.033142   
25%              2023-04-03 12:00:00      40.939259       0.903862   
50%              2023-06-29 12:00:00      41.665867       1.703217   
75%              2023-10-02 12:00:00      42.317552       2.527702   
max              2024-01-01 23:00:00      43.028343       3.469818   
std                              NaN       0.779556       0.948098   

           sconcno2  
count  3.320000e+05  
mean   8.617962e+33  
min    8.777442e-06  
25%    3.175537e-04  
50%    6.749267e-04  
75%    1.526916e-03  
max    9.969210e+36  
std    2.929851e+35  
time
2023-12-31 18:00:00    55
2023-01-02 18:00:0

In [4]:
directorio = r'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/'

# Listar los archivos .nc de la carpeta
archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.nc')]

# Lista para almacenar todos los DataFrames generados
lista_dataframes = []
def files_añadir(nc_data):
    # Seleccionar 1000 filas aleatorias
    directorio = r'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/'

    # Listar los archivos .nc de la carpeta
    archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.nc')]

    # Lista para almacenar todos los DataFrames generados
    lista_dataframes = []
    def files_añadir(nc_data):
        # Seleccionar 1000 filas aleatorias
        filas_con_valor_mayor_1 = nc_data.loc[(nc_data['sconcno2'] > 1) & (nc_data['sconcno2'] < 100)]
        global NO2
        NO2 = pd.concat([NO2, filas_con_valor_mayor_1], axis=0)

    # Recorre todos los archivos .nc de la carpeta
    for archivo in archivos:
        ruta_archivo = os.path.join(directorio, archivo)
        print(f"Procesando archivo: {archivo}")
        
        # Abrir el archivo NetCDF
        nc_data = xr.open_dataset(ruta_archivo)
        nc_data = nc_data.to_dataframe().reset_index()
        nc_data['time'] = pd.to_datetime(nc_data['time'])
        nc_data = nc_data.drop(columns=['y', 'x', 'lev'])
        
        sampled_data = files_añadir(nc_data)
    global NO2
    

# Recorre todos los archivos .nc de la carpeta
for archivo in archivos:
    ruta_archivo = os.path.join(directorio, archivo)
    print(f"Procesando archivo: {archivo}")
    
    # Abrir el archivo NetCDF
    nc_data = xr.open_dataset(ruta_archivo)
    nc_data = nc_data.to_dataframe().reset_index()
    nc_data['time'] = pd.to_datetime(nc_data['time'])
    nc_data = nc_data.drop(columns=['y', 'x', 'lev'])
    
    sampled_data = files_añadir(nc_data)
        
        

Procesando archivo: sconcno2_2023010100.nc
Procesando archivo: sconcno2_2023010200.nc
Procesando archivo: sconcno2_2023010300.nc
Procesando archivo: sconcno2_2023010400.nc
Procesando archivo: sconcno2_2023010500.nc
Procesando archivo: sconcno2_2023010600.nc
Procesando archivo: sconcno2_2023010700.nc
Procesando archivo: sconcno2_2023010800.nc
Procesando archivo: sconcno2_2023010900.nc
Procesando archivo: sconcno2_2023011000.nc
Procesando archivo: sconcno2_2023011100.nc
Procesando archivo: sconcno2_2023011200.nc
Procesando archivo: sconcno2_2023011400.nc
Procesando archivo: sconcno2_2023011500.nc
Procesando archivo: sconcno2_2023011600.nc
Procesando archivo: sconcno2_2023011700.nc
Procesando archivo: sconcno2_2023011800.nc
Procesando archivo: sconcno2_2023011900.nc
Procesando archivo: sconcno2_2023012000.nc
Procesando archivo: sconcno2_2023012200.nc
Procesando archivo: sconcno2_2023012300.nc
Procesando archivo: sconcno2_2023012500.nc
Procesando archivo: sconcno2_2023012600.nc
Procesando 

In [5]:
print(NO2.dtypes)

print(NO2.describe())

for i in NO2.columns:
    print(NO2[i].value_counts())

time                 datetime64[ns]
lat                         float64
lon                         float64
sconcno2                    float64
Lambert_conformal           float64
dtype: object
                                time           lat           lon  \
count                        1408972  1.408972e+06  1.408972e+06   
mean   2023-08-09 19:46:21.411696128  4.163907e+01  1.724602e+00   
min              2023-01-01 00:00:00  4.023144e+01 -1.190186e-03   
25%              2023-08-20 23:00:00  4.096356e+01  8.944397e-01   
50%              2023-08-21 15:00:00  4.164559e+01  1.721710e+00   
75%              2023-08-21 21:00:00  4.231059e+01  2.555359e+00   
max              2024-01-01 23:00:00  4.303321e+01  3.521942e+00   
std                              NaN  7.755333e-01  9.627296e-01   

           sconcno2  Lambert_conformal  
count  1.408972e+06       1.076972e+06  
mean   7.622168e+36      -2.147484e+09  
min    8.777442e-06      -2.147484e+09  
25%    9.969210e+36      -2.1

In [1]:
import os
import pandas as pd
import xarray as xr
import random
def extract_random_rows_v3(nc_data, num_rows=1000):
    """ Extraer 1000 filas aleatorias de las columnas time, lat, lon y sconcno2. """
    # Convertir el dataset de xarray a DataFrame y seleccionar solo las columnas necesarias
    df = nc_data[['time', 'lat', 'lon', 'sconcno2']].to_dataframe().reset_index()
    df['time'] = pd.to_datetime(df['time'])
    filas_horas_8 = df.loc[df['time'].dt.time == pd.to_datetime('8:00:00').time()]
    filas_horas_8['sconcno2'] = filas_horas_8['sconcno2'] * 1912.5
    # Seleccionar 1000 filas aleatorias
    return filas_horas_8


# Ruta de la carpeta donde se encuentran los archivos .nc
directorio = r'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/'

# Listar los archivos .nc de la carpeta
archivos = [archivo for archivo in os.listdir(directorio) if archivo.endswith('.nc')]

# Lista para almacenar todos los DataFrames generados
lista_dataframes = []

pas=random.randint(20, 30)
# Recorre todos los archivos .nc de la carpeta
for i in range(0, len(archivos), pas):
    archivo = archivos[i]
    try:
        ruta_archivo = os.path.join(directorio, archivo)
        print(f"Procesando archivo: {archivo}")
        
        # Abrir el archivo NetCDF
        nc_data = xr.open_dataset(ruta_archivo)
        
        # Extraer 1000 filas aleatorias
        sampled_data = extract_random_rows_v3(nc_data)
        
        # Agregar el DataFrame a la lista
        lista_dataframes.append(sampled_data)
        
        # Cerrar el dataset para liberar memoria
        nc_data.close()
        
        print(f"Archivo procesado correctamente: {archivo} | Filas extraídas: {sampled_data.shape[0]}")
    except Exception as e:
        print(f"Error al procesar {archivo}: {e}")

# Concatenar todos los DataFrames
data_final = pd.concat(lista_dataframes, ignore_index=True)
data_final = data_final.drop(columns=['x', 'y', 'lev'])
# Guardar el DataFrame final como CSV
output_path = 'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/NO2_final.csv'
data_final.to_csv(output_path, index=False)
print(f"Archivo CSV guardado en: {output_path}")

# Mostrar las primeras filas para verificar
print("Vista previa de los primeros registros:")
print(data_final.head())

# Mostrar información general del DataFrame
print(f"Total de filas: {data_final.shape[0]}")
print(f"Columnas del DataFrame: {data_final.columns}")


Procesando archivo: sconcno2_2023010100.nc


C:\Users\jiahu\AppData\Local\Temp\ipykernel_4140\1289764378.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas_horas_8['sconcno2'] = filas_horas_8['sconcno2'] * 1912.5


Archivo procesado correctamente: sconcno2_2023010100.nc | Filas extraídas: 165688
Procesando archivo: sconcno2_2023012700.nc


C:\Users\jiahu\AppData\Local\Temp\ipykernel_4140\1289764378.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas_horas_8['sconcno2'] = filas_horas_8['sconcno2'] * 1912.5


Archivo procesado correctamente: sconcno2_2023012700.nc | Filas extraídas: 165688
Procesando archivo: sconcno2_2023021900.nc


...

Archivo procesado correctamente: sconcno2_2023110100.nc | Filas extraídas: 165688
Procesando archivo: sconcno2_2023112800.nc


C:\Users\jiahu\AppData\Local\Temp\ipykernel_4140\1289764378.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas_horas_8['sconcno2'] = filas_horas_8['sconcno2'] * 1912.5


Archivo procesado correctamente: sconcno2_2023112800.nc | Filas extraídas: 165688
Procesando archivo: sconcno2_2023122200.nc


C:\Users\jiahu\AppData\Local\Temp\ipykernel_4140\1289764378.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filas_horas_8['sconcno2'] = filas_horas_8['sconcno2'] * 1912.5


Archivo procesado correctamente: sconcno2_2023122200.nc | Filas extraídas: 165688
Archivo CSV guardado en: C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/NO2_final.csv
Vista previa de los primeros registros:
                 time        lat       lon  sconcno2
0 2023-01-01 08:00:00  40.361107 -0.001190  0.632858
1 2023-01-01 08:00:00  40.360806  0.010620  0.653797
2 2023-01-01 08:00:00  40.360512  0.022430  0.711899
3 2023-01-01 08:00:00  40.360210  0.034241  0.701332
4 2023-01-01 08:00:00  40.359894  0.046051  0.641413
Total de filas: 2485320
Columnas del DataFrame: Index(['time', 'lat', 'lon', 'sconcno2'], dtype='object')


In [2]:
NO2_12 = pd.read_csv('C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/CALIOPE/NO2/NO2_final.csv')
NO2_12['time'] = pd.to_datetime(NO2_12['time'])
print(NO2_12.dtypes)

print(NO2_12.describe())
for i in NO2_12.columns:
    print(NO2_12[i].value_counts())

time        datetime64[ns]
lat                float64
lon                float64
sconcno2           float64
dtype: object
                                time           lat           lon      sconcno2
count                        2485320  2.485320e+06  2.485320e+06  2.485320e+06
mean   2023-06-27 13:35:59.999999744  4.163910e+01  1.726889e+00  3.250757e+00
min              2023-01-01 08:00:00  4.023144e+01 -1.190186e-03  1.656196e-02
25%              2023-03-20 08:00:00  4.096942e+01  8.902740e-01  7.058631e-01
50%              2023-06-24 20:00:00  4.163914e+01  1.726807e+00  1.500392e+00
75%              2023-10-06 08:00:00  4.230889e+01  2.562340e+00  3.354762e+00
max              2023-12-23 08:00:00  4.303321e+01  3.521942e+00  1.420376e+02
std                              NaN  7.742891e-01  9.671841e-01  5.716578e+00
time
2023-01-01 08:00:00    82844
2023-01-02 08:00:00    82844
2023-01-27 08:00:00    82844
2023-01-28 08:00:00    82844
2023-02-19 08:00:00    82844
2023-02-20 08:00:

In [8]:
import geopandas as gpd

# Ruta al archivo .shp (Geopandas también lee el .dbf)
ruta_archivo_shp = 'C:/Users/jiahu/OneDrive/Escritorio/AI3/Bitsxm/Dades/OpenTransportMap/Barcelona/roadlinks_ES511.shp'

# Cargar los datos del Shapefile (se incluye automáticamente el archivo .dbf)
gdf = gpd.read_file(ruta_archivo_shp)

# Convertir a DataFrame de Pandas
#df = gdf.drop(columns='geometry')  # Eliminar la columna de geometría si no la necesitas

# Mostrar las primeras filas
print(gdf.head())


   inspireid   sourceid beginlifes endlifespa  validfrom validto fictitious  \
0   54769333  198794320 2015-10-21        NaT 2015-10-21     NaT          0   
1   54800801  198935562 2015-10-21        NaT 2015-10-21     NaT          0   
2   54802891  198944072 2015-10-21        NaT 2015-10-21     NaT          0   
3   54808949  198965364 2015-10-21        NaT 2015-10-21     NaT          0   
4   54808951  198965365 2015-10-21        NaT 2015-10-21     NaT          0   

        direction  fromroadno  toroadnode  ... speedlimit capacity maximuheig  \
0  bothDirections    33620027    39891066  ...          0     2000        0.0   
1  bothDirections    36092175    39916521  ...         50     4000        0.0   
2     inDirection    39918239    39918240  ...          0     3000        0.0   
3  bothDirections    39923308    39923309  ...          0     2000        0.0   
4  bothDirections    39923311    39923312  ...          0     2000        0.0   

  maximumtot maximumwid  vehicletyp  z